## Коллокации

Коллокации - это устойчивые выражения, состоящие из двух и более слов. Устойчивые - значит, что они часто используются вместе.
Сегодня мы попробуем различные способы извлечения коллокаций из текстов

## Часть 0 : кастомные функции

In [1]:
# ! pip3 install razdel

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


[что такое razdel?](https://pypi.org/project/razdel/)

In [2]:
# традиционная ячейка импортов
import itertools
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from pymorphy2 import MorphAnalyzer
from collections import Counter, defaultdict
import numpy as np
import re
from string import punctuation

# раскомментьте строчки ниже, если NLTK вызывает проблемы при импорте

# import ssl
# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context

from nltk.corpus import stopwords

In [3]:
stops = set(stopwords.words('russian') + ["это", "весь"]) # определили список стоп-слов
morph = MorphAnalyzer()

# напишем несколько функций для предобработки текста

# нормализация текста
def normalize(text):
    
    tokens = re.findall('[а-яёa-z0-9]+', text.lower())
    normalized_text = [morph.parse(word)[0].normal_form for word \
                                                            in tokens]
    
    normalized_text = [word for word in normalized_text if len(word) > 2 \
                       and word not in stops]
    
    return normalized_text

# обработка текста
def preprocess(text):
    sents = sentenize(text) # разбили текст на предложения
    return [normalize(sent.text) for sent in sents]
# возвращаем список нормализованных предложений

def ngrammer(tokens, stops, n=2):
    ngrams = []
    tokens = [token for token in tokens if token not in stops]
    # добавляем все слова текста кроме стоп-слов
    for i in range(0,len(tokens)-n+1):
        # для каждого элемента списка, включительно, минус окно:
        ngrams.append(tuple(tokens[i:i+n]))
        # присоединяем элемент и все соседние в зависимости от размера окна
    return ngrams # возвращаем список таких кортежей

Предобработаем почти также, только теперь нам не нужны тэги начала и конца.

попробуем на каком-нибудь тестовом тексте, например, [про корги](https://ru.wikipedia.org/wiki/%D0%92%D0%B5%D0%BB%D1%8C%D1%88-%D0%BA%D0%BE%D1%80%D0%B3%D0%B8):

In [4]:
corpus = open('corgi.txt', encoding='utf-8').read()
# измените путь, если ваш датасет лежит в другом месте

теперь предобработаем наш корпус заранее написанной функцией *(может занять некоторое время)*

In [5]:
corpus = preprocess(corpus)

corpus -- это список списков. Давайте на него посмотрим:

In [6]:
print(len(corpus))

display(corpus[30:35]) 

82


[['контрастный',
  'обводка',
  'пасти',
  'многий',
  'казаться',
  'пемброк',
  'улыбаться'],
 ['глаз', 'карий'],
 ['красивый',
  'крепкий',
  'корпус',
  'крепкий',
  'короткий',
  'лапа',
  'выраженный',
  'угол'],
 ['считаться',
  'пемброк',
  'отличаться',
  'кардиган',
  'отсутствие',
  'хвост',
  'однако',
  'ошибка'],
 ['пемброк', 'рождаться', 'куцехвостый', 'хвост', 'купировать']]

В списке много всяких чисел, однобуквеных слов и стоп-слов.  Добавим какие-нибудь ограничения к коду выше, чтобы биграммы получались почище.

In [7]:
word_counter = Counter()

for sent in corpus:
    word_counter.update(ngrammer(sent, n=2, stops=stops))
    
#метод .update() (похож на традиционный .update() у словарей) обновляет показатели счетчика, а значит, и частоты

подробнее про Counter: [небольшой тьюториал](https://pythonworld.ru/moduli/modul-collections.html) и [официальная документация](https://docs.python.org/3/library/collections.html)

In [8]:
word_counter.most_common(10)

[(('вельша', 'корги'), 20),
 (('корги', 'пемброк'), 8),
 (('корги', 'относиться'), 3),
 (('корги', 'кардиган'), 3),
 (('пемброк', 'кардиган'), 3),
 (('рыжий', 'белый'), 3),
 (('пастуший', 'собака'), 2),
 (('англ', 'welsh'), 2),
 (('welsh', 'corgi'), 2),
 (('получить', 'распространение'), 2)]

В списке есть коллокации, которые попали туда, т.к. одно слово встречается в разных контекстах. 

Однако нас интересуют случаи, когда одни и те же слова встречаются вместе. Для этого мы можем придумать какие-нибудь формулы, учитывающие частоты слов по отдельности и общую частоту.

Самый простой способ - взять количество упоминаний какой-то биграммы и поделить на сумму количеств упоминаний слов по отдельности. 

$\frac{частота (биграмма)}{частота(слово1)+частота(слово2)}$

Такая формула называется PMI ([здесь подробнее](https://en.wikipedia.org/wiki/Pointwise_mutual_information))

In [9]:
# напишем свою формулу подсчета pmi
def scorer_simple(word_count_a, word_count_b, bigram_count, *args):
    try:
        score = bigram_count/((word_count_a+word_count_b))
    
    except ZeroDivisionError:
        return 0
    
    return score

Сделаем функцию, которая будет делать счетчики для слов и биграмм.

In [10]:
def collect_stats(corpus, stops):
    ## соберем статистики для отдельных слов и биграмм
    
    unigrams = Counter()
    bigrams = Counter()
    
    for sent in corpus:
        unigrams.update(sent)
        bigrams.update(ngrammer(sent, stops, 2))
    
    return unigrams, bigrams

И функцию, которая пройдет по всем биграммам и вычислит для них нашу метрику.

In [11]:
def score_bigrams(unigrams, bigrams, scorer, threshold=-100000, min_count=1):
    # посчитаем метрику для каждого нграмма
    
    bigram2score = Counter()
    len_vocab = len(unigrams)
    for bigram in bigrams:
        score = scorer(unigrams[bigram[0]], unigrams[bigram[1]], 
                       bigrams[bigram], len_vocab, min_count)
        
        # если метрика выше порога, добавляем в словарик
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

In [12]:
unigrams, bigrams = collect_stats(corpus, stops)

In [13]:
bigram2score = score_bigrams(unigrams, bigrams, scorer_simple)

Проблема с таким подходом в том, что на самом верху окажутся слова, которые встречают по одному разу.

In [14]:
bigram2score.most_common(10)

[(('англ', 'welsh'), 0.5),
 (('наиболее', 'популярный'), 0.5),
 (('завезти', 'кельт'), 0.5),
 (('кельт', 'освоение'), 0.5),
 (('вывести', 'пембрукшир'), 0.5),
 (('пембрукшир', 'предположительно'), 0.5),
 (('история', 'xiii'), 0.5),
 (('достоверно', 'неизвестно'), 0.5),
 (('теория', 'счёт'), 0.5),
 (('cur', 'смотреть'), 0.5)]

Поэтому можно немного переделать оценивающую функцию, добавив минимальное число вхождений для биграммы:

In [15]:
def scorer(word_count_a, word_count_b, bigram_count, len_vocab, min_count):
    try:
        score = ((bigram_count - min_count) / ((word_count_a + word_count_b)))
    except ZeroDivisionError:
        return 0
    
    return score

In [16]:
bigram2score = score_bigrams(unigrams, bigrams, scorer, min_count=20)

In [17]:
bigram2score.most_common(10)

[(('вельша', 'корги'), 0.0),
 (('корги', 'пемброк'), -0.23076923076923078),
 (('порода', 'корги'), -0.3877551020408163),
 (('корги', 'кардиган'), -0.40476190476190477),
 (('собака', 'вельша'), -0.4222222222222222),
 (('собака', 'порода'), -0.43902439024390244),
 (('корги', 'относиться'), -0.4594594594594595),
 (('порода', 'собака'), -0.4634146341463415),
 (('год', 'корги'), -0.475),
 (('корги', 'свой'), -0.475)]

*А [по этой ссылке](http://www.scielo.org.mx/scielo.php?script=sci_arttext&pid=S1405-55462016000300327#t1) можно прочитать про другие метрики*

## Часть 2: Все готовое

Писать все это самому конечно не обязательно. Попробуем использовать уже существующие специальные функции из библиотек

### Gensim

Удобно пользоваться [phraser из gensim'а](https://radimrehurek.com/gensim/models/phrases.html). Он собирает статистику по корпусу, а затем склеивает слова в биграммы.

In [18]:
# !pip3 install gensim

In [19]:
import gensim

In [20]:
# собираем статистики
ph = gensim.models.Phrases(corpus)

In [21]:
# преобразовывать можно и через ph, но так быстрее 
p = gensim.models.phrases.Phraser(ph)

По умолчанию там используется метрики из статьи про word2vec и ещё есть нормализованные pmi.
Если не нравятся функции оценки, то ему можно подать любую другую функцию. Интерфейс у функции там почти точно такой же как и у наших.

In [22]:
# собираем статистики по уже забиграммленному тексту
ph2 = gensim.models.Phrases(p[corpus])
p2 = gensim.models.phrases.Phraser(ph2)

In [23]:
p2[p[corpus[30]]]

['контрастный',
 'обводка',
 'пасти',
 'многий',
 'казаться',
 'пемброк',
 'улыбаться']

### NLTK
Тут больше метрик, но преборазователь слов в нграммы нужно написать самому.

In [24]:
import nltk
from nltk.collocations import * # звездочка говорит, что мы импортируем всё

из NLTK будем использовать модуль с коллокациями, [вот подробнее про него](https://www.nltk.org/howto/collocations.html)

In [25]:
bigram_measures = nltk.collocations.BigramAssocMeasures() 
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [26]:
finder2 = BigramCollocationFinder.from_documents(corpus)

In [27]:
finder3 = TrigramCollocationFinder.from_documents(corpus)

In [28]:
finder2.nbest(bigram_measures.likelihood_ratio, 20)

[('вельша', 'корги'),
 ('рыжий', 'белый'),
 ('корги', 'пемброк'),
 ('англ', 'welsh'),
 ('будущий', 'король'),
 ('герцог', 'йоркский'),
 ('дочь', 'элизабет'),
 ('грудной', 'клетка'),
 ('получить', 'распространение'),
 ('welsh', 'corgi'),
 ('тигровый', 'окрас'),
 ('1933', 'год'),
 ('свой', 'дочь'),
 ('королевский', 'семья'),
 ('британский', 'королевский'),
 ('короткий', 'лапа'),
 ('crufts', 'кинологический'),
 ('cur', 'смотреть'),
 ('eagle', 'коротко'),
 ('golden', 'eagle')]

In [29]:
finder3.nbest(trigram_measures.pmi, 20)

[('crufts', 'кинологический', 'мероприятие'),
 ('cur', 'смотреть', 'сторожить'),
 ('golden', 'eagle', 'коротко'),
 ('rozavel', 'golden', 'eagle'),
 ('аджилить', 'флайбола', 'соревнование'),
 ('актёр', 'дэниэлом', 'крейг'),
 ('благородный', 'очертание', 'прямой'),
 ('благородство', 'мощь', 'работоспособность'),
 ('важно', 'видеть', 'довольный'),
 ('вальхунд', 'вестготашпиц', 'исландский'),
 ('ввести', 'запрет', 'купирование'),
 ('величество', 'монтить', 'уиллоу'),
 ('вероятно', 'послужить', 'шведский'),
 ('видеть', 'довольный', 'пёс'),
 ('вместе', 'хозяйка', 'актёр'),
 ('возбудимый', 'живой', 'чуткий'),
 ('возникать', 'проблема', 'здоровье'),
 ('вывести', 'пембрукшир', 'предположительно'),
 ('голос', 'увидеть', 'знакомый'),
 ('двор', 'жить', 'поколение')]

### Sklearn

Sklearn напрямую не предназначен для этого, но из него тоже можно вытаскивать устойчивые нграммы. Tfidf подходит как метрика.

In [30]:
# !pip3 install scikit learn

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer

А вот рассказ о том, [что такое TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [32]:
# по умолчанию векторайзер сам токенизирует, поэтому проще склеить токены через пробел
texts = [' '.join(sent) for sent in corpus]

In [33]:
tfidf = TfidfVectorizer(min_df=2, max_df=0.7, max_features=100,
                       ngram_range=(2,2))

In [34]:
tfidf.fit(texts)

TfidfVectorizer(max_df=0.7, max_features=100, min_df=2, ngram_range=(2, 2))

In [35]:
# словарь со словами и индекасами
tfidf.vocabulary_

{'пастуший собака': 16,
 'корги относиться': 11,
 'вельша корги': 6,
 'корги кардиган': 10,
 'англ welsh': 2,
 'welsh corgi': 1,
 'корги пемброк': 12,
 'получить распространение': 20,
 '1933 год': 0,
 'герцог йоркский': 7,
 'будущий король': 5,
 'щенок вельша': 26,
 'свой дочь': 22,
 'дочь элизабет': 9,
 'пемброк кардиган': 17,
 'тигровый окрас': 25,
 'грудной клетка': 8,
 'пемброк рыжий': 19,
 'рыжий белый': 21,
 'белый окрас': 3,
 'короткий лапа': 15,
 'свой хозяин': 23,
 'корги склонный': 13,
 'пемброк кличка': 18,
 'британский королевский': 4,
 'королевский семья': 14,
 'собака порода': 24}

In [36]:
# массив с метриками, можно достать по индексу из словаря
tfidf.idf_

array([4.32022832, 4.32022832, 4.32022832, 4.32022832, 4.32022832,
       4.32022832, 2.37431817, 4.32022832, 4.32022832, 4.32022832,
       4.03254625, 4.03254625, 3.22161603, 4.32022832, 4.32022832,
       4.32022832, 4.32022832, 4.03254625, 4.32022832, 4.32022832,
       4.32022832, 4.03254625, 4.32022832, 4.32022832, 4.32022832,
       4.32022832, 4.32022832])

In [37]:
word2idf = []

for word, i in tfidf.vocabulary_.items():
    word2idf.append((tfidf.idf_[i], word))

In [38]:
sorted(word2idf, reverse=True)

[(4.320228319128488, 'щенок вельша'),
 (4.320228319128488, 'тигровый окрас'),
 (4.320228319128488, 'собака порода'),
 (4.320228319128488, 'свой хозяин'),
 (4.320228319128488, 'свой дочь'),
 (4.320228319128488, 'получить распространение'),
 (4.320228319128488, 'пемброк рыжий'),
 (4.320228319128488, 'пемброк кличка'),
 (4.320228319128488, 'пастуший собака'),
 (4.320228319128488, 'короткий лапа'),
 (4.320228319128488, 'королевский семья'),
 (4.320228319128488, 'корги склонный'),
 (4.320228319128488, 'дочь элизабет'),
 (4.320228319128488, 'грудной клетка'),
 (4.320228319128488, 'герцог йоркский'),
 (4.320228319128488, 'будущий король'),
 (4.320228319128488, 'британский королевский'),
 (4.320228319128488, 'белый окрас'),
 (4.320228319128488, 'англ welsh'),
 (4.320228319128488, 'welsh corgi'),
 (4.320228319128488, '1933 год'),
 (4.0325462466767075, 'рыжий белый'),
 (4.0325462466767075, 'пемброк кардиган'),
 (4.0325462466767075, 'корги относиться'),
 (4.0325462466767075, 'корги кардиган'),
 (

Преобразовать отдельный текст в список нграммов сложнее, но можно (правда порядок не получится сохранить). 